# Comparing channels

- R: sequential
- G: reverse complementary
- B: not match at all

In [1]:
import ppscore
import pandas
import os
import numpy
from scipy.stats import spearmanr
from IPython.core.display import display, HTML
from ete3 import Tree

/tmp/ipykernel_29639/499083497.py:6: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [2]:
tree_path = "../../data/trees"

In [3]:
def read_and_compare(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Euclidean Distance from distance matrices

In [4]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9085,1.1081,0.0,0.8937,1.0098,0.4040,0.3869,0.3705,0.3708,0.3923,1.0630,1.1279,1.1527,1.1396,2.3386
orthologs_myoglobin,0.9196,1.0228,0.0,0.9044,0.9527,0.4211,0.4276,0.3084,0.3213,0.3419,1.0055,1.0761,1.0707,1.1520,2.3185
orthologs_neuroglobin,1.8043,5.5680,0.0,1.1514,5.4438,1.0372,1.0310,3.2488,3.2313,3.2114,5.5988,5.5166,5.4733,1.7182,5.5936
orthologs_cytoglobin,6.3417,7.3329,0.0,3.1095,4.9159,1.9077,1.3956,3.9678,3.3164,4.9814,5.0731,9.9692,7.5790,3.2222,9.3160
orthologs_androglobin,8.2855,10.9109,0.0,2.9601,9.7685,3.3699,2.9019,6.8149,6.5207,7.3436,10.0188,12.2551,11.0215,1.5366,11.5645


In [5]:
def read_and_linear_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(spearmanr(control_df.values.flatten(), df.values.flatten())[0], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Linear Correlation from distance matrices

In [7]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_linear_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9999,0.9972,1.0,0.9998,0.9970,0.9972,0.9968,0.9975,0.9973,0.9972,0.9970,0.9972,0.9968,0.9190
orthologs_myoglobin,0.9999,0.9925,1.0,0.9998,0.9957,0.9963,0.9914,0.9937,0.9948,0.9941,0.9959,0.9963,0.9914,0.9489
orthologs_neuroglobin,0.9999,0.9945,1.0,0.9992,0.9766,0.9948,0.9856,0.9952,0.9778,0.9940,0.9763,0.9948,0.9856,0.9538
orthologs_cytoglobin,0.7775,0.5939,1.0,0.8623,0.4996,0.4896,0.5775,0.6044,0.5556,0.6096,0.5001,0.4896,0.5775,0.5906
orthologs_androglobin,0.9054,0.7332,1.0,0.9769,0.7373,0.2940,0.6960,0.7393,0.7343,0.6851,0.7367,0.2940,0.6960,0.7344


In [8]:
def read_and_correlate(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_df = pandas.read_csv(f"{path}/Control with Clustal Omega.csv", index_col=0)
    for file in os.listdir(path):
        if file.endswith(".csv"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                df = pandas.read_csv(f"{path}/{file}", index_col=0)
                result_dict[basename] = round(ppscore.score(
                    pandas.DataFrame({"x": control_df.values.flatten(), "y": df.values.flatten()}), "x", "y")["ppscore"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Predictive Power Score from distance matrices

In [9]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_correlate(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,full,Control with Clustal Omega,Local with Smith–Waterman,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_mean
dataset,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.9953,0.9651,0.9968,0.9944,0.9650,0.9629,0.9654,0.9648,0.9673,0.9668,0.9646,0.9629,0.9654,0.8945
orthologs_myoglobin,0.9954,0.9494,0.9968,0.9925,0.9564,0.9461,0.9341,0.9528,0.9539,0.9394,0.9561,0.9461,0.9341,0.8670
orthologs_neuroglobin,0.9876,0.9650,0.9964,0.9897,0.9640,0.9660,0.9632,0.9651,0.9640,0.9650,0.9640,0.9660,0.9632,0.9325
orthologs_cytoglobin,0.8524,0.7637,0.9913,0.8584,0.7056,0.7841,0.7702,0.7611,0.7374,0.7790,0.7056,0.7841,0.7702,0.7901
orthologs_androglobin,0.9773,0.9315,0.9977,0.9799,0.9390,0.8677,0.9253,0.9342,0.9348,0.9031,0.9390,0.8677,0.9253,0.8965


In [10]:
def read_and_tree_compare(dataset, channel="full", control="Control with Clustal Omega"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/{control}.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = round(result["norm_rf"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Robinson-foulds distance from generated Trees

In [11]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0000,0.0,0.1667,0.0,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0000,0.0,0.0833,0.0,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.6667,0.5,0.6667,0.0,0.6667,0.8333,0.6667,0.6667,0.5833,0.6667,0.6667,0.8333,0.6667,1.0,0.8333
orthologs_androglobin,0.8333,0.5,0.7500,0.0,0.9167,0.9167,0.7500,0.7500,0.7500,0.9167,0.9167,0.9167,0.7500,1.0,0.8333


In [12]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Global with Needleman-Wunsch"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0,0.0000,0.1667,0.0000,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0,0.0000,0.0833,0.0000,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.0,0.2500,0.5833,0.6667,0.7500,0.5833,0.5833,0.5833,0.5833,0.5833,0.7500,0.5833,0.5833,1.0,0.5833
orthologs_androglobin,0.0,0.4167,0.6667,0.8333,0.8333,0.5833,0.6667,0.6667,0.6667,0.6667,0.8333,0.5833,0.6667,1.0,0.6667


In [13]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare(dataset, channel, "Local with Smith–Waterman"))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,1.0,0.2500
orthologs_myoglobin,0.0000,0.0,0.1667,0.0,0.0833,0.1667,0.0833,0.0833,0.1667,0.1667,0.0833,0.1667,0.0833,1.0,0.1667
orthologs_neuroglobin,0.0000,0.0,0.0833,0.0,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,0.0833,1.0,0.2500
orthologs_cytoglobin,0.2500,0.0,0.5833,0.5,0.5833,0.6667,0.5833,0.5833,0.4167,0.5833,0.5833,0.6667,0.5833,1.0,0.6667
orthologs_androglobin,0.4167,0.0,0.5833,0.5,0.6667,0.6667,0.5833,0.5833,0.5833,0.6667,0.6667,0.6667,0.5833,1.0,0.5833


In [14]:
def read_and_tree_compare_branches(dataset, channel="full"):
    result_dict = {}
    path = f"{tree_path}/{channel}/{dataset}"
    control_tree = Tree(f"{path}/Control with Clustal Omega.nw", format=1)
    for file in os.listdir(path):
        if file.endswith(".nw"):
            basename = ".".join(file.split(".")[0:-1])
            try:
                tree = Tree(f"{path}/{file}", format=1)
                result = control_tree.compare(tree, unrooted=True)
                result_dict[basename] = 1.0 - round(result["source_edges_in_ref"], 4)
            except FileNotFoundError:
                pass
    result_df = pandas.DataFrame(result_dict, index=[dataset])
    result_df.rename(columns={"Structural Similarity Index Measure": channel}, inplace=True)
    result_df.index.name = "dataset"
    return result_df

## Compatibility branch score from generated Trees

In [15]:
channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
datasets = ["orthologs_hemoglobin_beta", "orthologs_myoglobin", "orthologs_neuroglobin", "orthologs_cytoglobin", "orthologs_androglobin"]
sum_dfs = []
for channel in channels:
    dfs = []
    for dataset in datasets:
        dfs.append(read_and_tree_compare_branches(dataset, channel))
    sum_dfs.append(pandas.concat(dfs))
pandas.concat(sum_dfs, axis=1).T.reset_index().drop_duplicates(["index"]).set_index("index").T

index,Global with Needleman-Wunsch,Local with Smith–Waterman,full,Control with Clustal Omega,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,,,,
orthologs_hemoglobin_beta,0.0000,0.0000,0.0000,0.0,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.0000,0.4615,0.1154
orthologs_myoglobin,0.0000,0.0000,0.0769,0.0,0.0385,0.0769,0.0385,0.0385,0.0769,0.0769,0.0385,0.0769,0.0385,0.4615,0.0769
orthologs_neuroglobin,0.0000,0.0000,0.0385,0.0,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.0385,0.4615,0.1154
orthologs_cytoglobin,0.3077,0.2308,0.3077,0.0,0.3077,0.3846,0.3077,0.3077,0.2692,0.3077,0.3077,0.3846,0.3077,0.4615,0.3846
orthologs_androglobin,0.3846,0.2308,0.3462,0.0,0.4231,0.4231,0.3462,0.3462,0.3462,0.4231,0.4231,0.4231,0.3462,0.4615,0.3846


In [16]:
def read_and_compare_channels(dataset):
    result_dict = {}
    dfs = {}
    channels = ("full", "red", "green", "blue", "red_green", "red_blue", "green_blue", "gray_r", "gray_g", "gray_b", "gray_max", "gray_mean")
    for channel in channels:
        result_dict[channel] = {}
        path = f"{tree_path}/{channel}/{dataset}"
        dfs[channel] = pandas.read_csv(f"{path}/Structural Similarity Index Measure.csv", index_col=0)
    for channel in channels:
        control_df = dfs[channel]
        for c, df in dfs.items():
            result_dict[channel][c] = round(numpy.sqrt(numpy.sum((control_df.values - df.values)**2)), 4)
    result_df = pandas.DataFrame.from_dict(result_dict, orient='index')
    result_df.index.name = "dataset"
    return result_df.style.background_gradient(axis=None, vmin=result_df.min().min(), vmax=result_df.max().max(), cmap="YlGnBu")

## Features Euclidean distance from distance matrices

In [17]:
for dataset in datasets:
    try:
        correlations = read_and_compare_channels(dataset)
    except FileNotFoundError:
        continue
    else:
        display(HTML(f"<center><h3>{dataset}</h3></center>"))
        display(correlations)        

,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,0.109300,1.483700,1.473300,0.759700,0.749000,0.725400,0.062000,0.059700,0.093000,2.233800,1.270300
red,0.109300,0.000000,1.382000,1.372200,0.657800,0.648400,0.625300,0.054100,0.131900,0.184700,2.132100,1.362700
green,1.483700,1.382000,0.000000,0.041700,0.724500,0.735800,0.758900,1.435900,1.501100,1.534800,0.750600,2.712000
blue,1.473300,1.372200,0.041700,0.000000,0.715700,0.724600,0.748100,1.426100,1.492000,1.522500,0.761300,2.704800
red_green,0.759700,0.657800,0.724500,0.715700,0.000000,0.046500,0.053200,0.711700,0.777500,0.814500,1.474800,1.998800
red_blue,0.749000,0.648400,0.735800,0.724600,0.046500,0.000000,0.031900,0.702200,0.769100,0.799700,1.485400,1.993100
green_blue,0.725400,0.625300,0.758900,0.748100,0.053200,0.031900,0.000000,0.679100,0.744700,0.776600,1.508800,1.969000
gray_r,0.062000,0.054100,1.435900,1.426100,0.711700,0.702200,0.679100,0.000000,0.086200,0.143600,2.186100,1.311200
gray_g,0.059700,0.131900,1.501100,1.492000,0.777500,0.769100,0.744700,0.086200,0.000000,0.125000,2.251700,1.245800


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,0.116300,1.421600,1.424300,0.733900,0.714400,0.696200,0.098700,0.112300,0.085300,2.159900,1.385400
red,0.116300,0.000000,1.356100,1.360400,0.670400,0.651200,0.635300,0.053300,0.159700,0.178900,2.094300,1.417700
green,1.421600,1.356100,0.000000,0.039000,0.688900,0.708200,0.728000,1.409300,1.478800,1.470800,0.739400,2.724700
blue,1.424300,1.360400,0.039000,0.000000,0.692200,0.710800,0.730900,1.413500,1.483500,1.472500,0.736200,2.732400
red_green,0.733900,0.670400,0.688900,0.692200,0.000000,0.041300,0.074300,0.723400,0.795400,0.786500,1.426800,2.058700
red_blue,0.714400,0.651200,0.708200,0.710800,0.041300,0.000000,0.053100,0.704200,0.775400,0.764500,1.446300,2.044100
green_blue,0.696200,0.635300,0.728000,0.730900,0.074300,0.053100,0.000000,0.688000,0.754500,0.744200,1.466400,2.031600
gray_r,0.098700,0.053300,1.409300,1.413500,0.723400,0.704200,0.688000,0.000000,0.124300,0.152400,2.147400,1.368400
gray_g,0.112300,0.159700,1.478800,1.483500,0.795400,0.775400,0.754500,0.124300,0.000000,0.116900,2.218200,1.326000


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,0.142100,4.743600,4.759300,2.351000,2.372200,2.386600,0.062100,0.087600,0.127600,7.092500,0.829200
red,0.142100,0.000000,4.612700,4.628400,2.220200,2.241400,2.256300,0.156000,0.137900,0.130300,6.961400,0.862300
green,4.743600,4.612700,0.000000,0.047400,2.392900,2.372100,2.357300,4.768600,4.698200,4.652100,2.349100,4.829300
blue,4.759300,4.628400,0.047400,0.000000,2.409300,2.387300,2.372800,4.784300,4.714400,4.667100,2.333500,4.841000
red_green,2.351000,2.220200,2.392900,2.409300,0.000000,0.067100,0.065300,2.376200,2.305800,2.261500,4.741900,2.512700
red_blue,2.372200,2.241400,2.372100,2.387300,0.067100,0.000000,0.035600,2.397200,2.328600,2.280700,4.720600,2.521600
green_blue,2.386600,2.256300,2.357300,2.372800,0.065300,0.035600,0.000000,2.412200,2.342100,2.295000,4.706100,2.533900
gray_r,0.062100,0.156000,4.768600,4.784300,2.376200,2.397200,2.412200,0.000000,0.132700,0.170600,7.117300,0.849200
gray_g,0.087600,0.137900,4.698200,4.714400,2.305800,2.328600,2.342100,0.132700,0.000000,0.142200,7.047300,0.837900


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,3.657700,5.982900,6.699700,3.468200,4.292600,2.562900,3.555600,3.119900,0.371600,10.170600,2.407900
red,3.657700,0.000000,4.015400,4.297700,2.076100,1.916900,2.872300,0.165900,6.771300,4.002500,7.475100,6.044500
green,5.982900,4.015400,0.000000,0.927100,2.580300,2.126600,3.494100,4.164000,8.533600,6.207300,4.266800,7.893800
blue,6.699700,4.297700,0.927100,0.000000,3.236900,2.561400,4.289900,4.458200,9.364300,6.945700,3.472800,8.707100
red_green,3.468200,2.076100,2.580300,3.236900,0.000000,0.966300,1.290000,2.161400,6.270100,3.728800,6.704200,5.583000
red_blue,4.292600,1.916900,2.126600,2.561400,0.966300,0.000000,2.250800,2.057000,7.190500,4.577000,5.962300,6.491200
green_blue,2.562900,2.872300,3.494100,4.289900,1.290000,2.250800,0.000000,2.891800,5.082400,2.746500,7.746900,4.425000
gray_r,3.555600,0.165900,4.164000,4.458200,2.161400,2.057000,2.891800,0.000000,6.670000,3.902300,7.640700,5.943700
gray_g,3.119900,6.771300,8.533600,9.364300,6.270100,7.190500,5.082400,6.670000,0.000000,2.781300,12.800400,0.814300


,full,red,green,blue,red_green,red_blue,green_blue,gray_r,gray_g,gray_b,gray_max,gray_mean
dataset,,,,,,,,,,,,
full,0.000000,3.183500,7.846300,8.105600,4.110600,4.511300,3.809000,3.134600,2.761300,0.353700,12.204700,1.948700
red,3.183500,0.000000,7.312300,7.245700,3.782200,3.544700,4.381400,0.251100,5.935200,3.522200,11.091100,5.083500
green,7.846300,7.312300,0.000000,0.806500,3.812900,3.780400,4.116500,7.554000,8.975600,7.912100,4.487800,8.319300
blue,8.105600,7.245700,0.806500,0.000000,4.001500,3.800800,4.502500,7.493300,9.441900,8.202900,4.101400,8.741700
red_green,4.110600,3.782200,3.812900,4.001500,0.000000,0.820300,1.150800,3.997900,5.775900,4.227600,8.096100,4.980600
red_blue,4.511300,3.544700,3.780400,3.800800,0.820300,0.000000,1.968700,3.782400,6.428700,4.677600,7.822700,5.599900
green_blue,3.809000,4.381400,4.116500,4.502500,1.150800,1.968700,0.000000,4.560800,4.962100,3.833000,8.574900,4.244900
gray_r,3.134600,0.251100,7.554000,7.493300,3.997900,3.782400,4.560800,0.000000,5.894300,3.477400,11.342200,5.054100
gray_g,2.761300,5.935200,8.975600,9.441900,5.775900,6.428700,4.962100,5.894300,0.000000,2.419400,13.463400,0.930200
